In [1]:
import pandas as pd
import numpy as np

In [2]:
gameInput = pd.read_csv("nhl-game-data/game.csv").dropna()
goalieInput = pd.read_csv("nhl-game-data/game_goalie_stats.csv")
# skaterStatsInput = pd.read_csv("nhl-game-data/game_skater_stats.csv")
# teamsStatsInput = pd.read_csv("nhl-game-data/game_teams_stats.csv")
# teamInfoInput = pd.read_csv("nhl-game-data/team_info.csv")

In [4]:
goalies = goalieInput.loc[:,['game_id','team_id','shots','saves']].groupby(['game_id','team_id']).sum().reset_index()
goalies['percentage'] = (goalies['saves'] / goalies['shots'])
print(type(goalies.loc[goalies['game_id'] == 2010020001]['percentage'].values[0]))

<class 'numpy.float64'>


In [5]:
goalies1 = pd.DataFrame(columns=['game_id', 'home_percentage', 'away_percentage'])
for index, row in gameInput.iterrows():
    if not goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['away_team_id'])].empty:
        awayPerc = goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['away_team_id'])]
    if not goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['home_team_id'])].empty:
        homePerc = goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['home_team_id'])]
    d = {'game_id': row['game_id'], 'home_percentage': homePerc['percentage'].values[0], 'away_percentage': awayPerc['percentage'].values[0]}
    goalies1.at[index, :] = d
goalies1

,game_id,home_percentage,away_percentage
0,2.01103e+09,0.884615,0.888889
1,2.01103e+09,0.911765,0.95
2,2.01103e+09,0.892857,0.870968
3,2.01103e+09,0.909091,0.928571
4,2.01103e+09,0.9,0.964286
...,...,...,...
11429,2.01803e+09,0.73913,0.931034
11430,2.01803e+09,0.913043,0.918919
11431,2.01803e+09,0.904762,0.974359
11432,2.01803e+09,0.870968,0.965517


In [6]:
gameInput1 = gameInput.join(goalies1, lsuffix='', rsuffix='_goalie')


In [7]:
gameAwayValues = gameInput1.assign(away_percentage=pd.to_numeric(gameInput1['away_percentage'], errors='coerce')).loc[:,['away_team_id', 'away_goals', 'home_goals', 'away_percentage', 'season']].groupby(['away_team_id', 'season']).mean().reset_index()
gameHomeValues = gameInput1.assign(home_percentage=pd.to_numeric(gameInput1['home_percentage'], errors='coerce')).loc[:,['home_team_id', 'away_goals', 'home_goals', 'home_percentage', 'season']].groupby(['home_team_id', 'season']).mean().reset_index()
# gameAwayValues
gameHomeValues
# gameInput1.loc[:,['away_team_id', 'away_percentage', 'season' ]]

,home_team_id,season,away_goals,home_goals,home_percentage
0,1,20102011,2.463415,2.292683,0.898303
1,1,20112012,2.461538,2.846154,0.906777
2,1,20122013,2.458333,2.750000,0.892649
3,1,20132014,2.243902,2.560976,0.916448
4,1,20142015,2.195122,2.219512,0.931411
...,...,...,...,...,...
260,53,20142015,3.243902,1.902439,0.907676
261,53,20152016,2.756098,2.975610,0.906778
262,53,20162017,2.853659,2.536585,0.918366
263,53,20172018,2.878049,2.487805,0.914878


In [14]:
dataSet1 = pd.DataFrame(columns=['season', 'date_time', 'game1home','game1away', 'game1isHome','game2home', \
                                 'game2away','game2isHome','game3home','game3away','game3isHome','game4home','game4away',\
                                 'game4isHome','game5home','game5away','game5isHome', 'home_perc', 'away_perc','home_team_id', 'away_team_id', \
                                 'home_scored_avg', 'home_conceded_avg', 'away_scored_avg', 'away_conceded_avg', \
                                 'total'])
# print(gameAwayValues.loc[(gameAwayValues['away_team_id'] == '4') & (gameAwayValues['season'] == '20102011')])
gameAwayValues = gameAwayValues.dropna()
gameHomeValues = gameHomeValues.dropna()
for index, row in gameInput1.iterrows():
    season = row['season'] - 10001
    if not gameAwayValues.loc[(gameAwayValues['away_team_id'] == row['away_team_id']) & (gameAwayValues['season'] == season)].empty:
        if not gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['home_team_id']) & (gameHomeValues['season'] == season)].empty:
            awayTeam = gameAwayValues.loc[(gameAwayValues['away_team_id'] == row['away_team_id']) & (gameAwayValues['season'] == season)]
            homeTeam = gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['home_team_id']) & (gameHomeValues['season'] == season)]
#             print(homeTeam, awayTeam)
            d = {'game_id': row['game_id'],\
                 'season': row['season'],\
                 'date_time': row['date_time'],\
                 'home_team_id': row['home_team_id'], \
                 'home_perc': homeTeam['home_percentage'].values[0], \
                 'away_perc': awayTeam['away_percentage'].values[0], \
                 'home_scored_avg': homeTeam['home_goals'].values[0], \
                 'home_conceded_avg': homeTeam['away_goals'].values[0], \
                 'away_team_id': row['away_team_id'], \
                 'away_scored_avg': awayTeam['away_goals'].values[0], \
                 'away_conceded_avg': awayTeam['home_goals'].values[0], \
                 'total': row['home_goals'] + row['away_goals']}
            for i in range(1,6):
                d['game' + str(i) + 'home'] = 0
                d['game' + str(i) + 'away'] = 0
                d['game' + str(i) + 'isHome'] = 1
            if not gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5).empty:
                previousGames = gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5)
                i = 1
                for index1, row1 in previousGames.iterrows():
                    if (row1.home_team_id == row.home_team_id):
                        d['game' + str(i) + 'home'] = row1['home_goals']
                        d['game' + str(i) + 'away'] = row1['away_goals']
                        d['game' + str(i) + 'isHome'] = 1
                    else:
                        d['game' + str(i) + 'home'] = row1['away_goals']
                        d['game' + str(i) + 'away'] = row1['home_goals']
                        d['game' + str(i) + 'isHome'] = 0
                    i += 1
                    
            dataSet1.at[index, :] = d
#             d = {'game_id': row['game_id'], 
#                  'season': row['season'], 
#                  'date_time': row['date_time'], 
#                  'home_team_id': row['home_team_id'], 
#                  '1_scored': homeTeam['home_goals'].values[0], 
#                  '1_agg': homeTeam['away_goals'].values[0], 
#                  '1_perc': homeTeam['home_percentage'].values[0], 
#                  'away_team_id': row['away_team_id'], 
#                  '2_scored': awayTeam['away_goals'].values[0], 
#                  '2_agg': awayTeam['home_goals'].values[0],
#                  '2_perc': awayTeam['away_percentage'].values[0]
#                 }
#             dataSet2.at[index, :] = d

In [15]:
dataSet1 = dataSet1.sort_values(by=['date_time'])
dataSet1

,season,date_time,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,...,game5isHome,home_perc,away_perc,home_team_id,away_team_id,home_scored_avg,home_conceded_avg,away_scored_avg,away_conceded_avg,total
11049,20112012,2011-10-06,0,2,0,1,4,0,5,4,...,0,0.908803,0.915399,10,8,2.68293,2.90244,2.53488,2.65116,2
9192,20112012,2011-10-06,1,2,1,3,0,0,2,1,...,0,0.93641,0.910504,6,4,3.14815,2.16667,3,2.84091,3
10684,20112012,2011-10-07,2,3,0,2,4,0,5,4,...,0,0.905153,0.925626,26,3,2.77273,2.54545,2.45238,2.35714,5
10477,20112012,2011-10-07,4,3,0,2,1,0,1,2,...,0,0.901833,0.895013,17,9,3.19565,3.08696,2.33333,3.25641,8
9312,20112012,2011-10-07,4,0,1,4,3,0,1,4,...,0,0.895866,0.920074,29,18,2.65854,3.07317,2.5,2.45455,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,20182019,2019-06-02,2,4,1,2,4,0,3,2,...,0,0.909999,0.914795,19,6,2.80488,2.53659,2.95652,2.69565,9
11430,20182019,2019-06-04,2,4,0,3,2,0,2,7,...,0,0.909999,0.914795,19,6,2.80488,2.53659,2.95652,2.69565,6
11431,20182019,2019-06-07,4,2,1,2,3,1,7,2,...,0,0.908886,0.915808,6,19,3.70213,2.6383,2.71795,2.84615,3
11432,20182019,2019-06-10,3,2,0,2,7,1,4,2,...,0,0.909999,0.914795,19,6,2.80488,2.53659,2.95652,2.69565,6


In [16]:
dataSetFinal = dataSet1.drop('date_time',1)
dataTest1 = dataSetFinal.loc[dataSetFinal.season == 20182019]
dataTest1

,season,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,...,game5isHome,home_perc,away_perc,home_team_id,away_team_id,home_scored_avg,home_conceded_avg,away_scored_avg,away_conceded_avg,total
7616,20182019,1,4,0,5,4,0,2,0,1,...,0,0.921074,0.894555,10,8,3.45455,2.61364,2.25,3.525,5
7617,20182019,5,3,1,1,4,1,5,3,1,...,0,0.915929,0.914795,15,6,3.40385,2.55769,2.95652,2.69565,7
7622,20182019,3,2,1,3,4,1,4,2,0,...,0,0.900432,0.897982,12,2,2.68293,2.97561,3.05,3.55,3
7623,20182019,4,2,1,2,1,0,4,3,0,...,1,0.911178,0.928944,3,18,3.07317,3.09756,3.15217,2.52174,5
7624,20182019,4,3,0,4,3,1,7,8,0,...,1,0.890725,0.892391,9,16,3.04878,3.73171,2.71795,3.46154,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,20182019,2,4,1,2,4,0,3,2,0,...,0,0.909999,0.914795,19,6,2.80488,2.53659,2.95652,2.69565,9
11430,20182019,2,4,0,3,2,0,2,7,1,...,0,0.909999,0.914795,19,6,2.80488,2.53659,2.95652,2.69565,6
11431,20182019,4,2,1,2,3,1,7,2,0,...,0,0.908886,0.915808,6,19,3.70213,2.6383,2.71795,2.84615,3
11432,20182019,3,2,0,2,7,1,4,2,1,...,0,0.909999,0.914795,19,6,2.80488,2.53659,2.95652,2.69565,6


In [17]:
dataTrain1 = dataSetFinal.loc[dataSetFinal.season != 20182019]
dataTrain1

,season,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,...,game5isHome,home_perc,away_perc,home_team_id,away_team_id,home_scored_avg,home_conceded_avg,away_scored_avg,away_conceded_avg,total
11049,20112012,0,2,0,1,4,0,5,4,0,...,0,0.908803,0.915399,10,8,2.68293,2.90244,2.53488,2.65116,2
9192,20112012,1,2,1,3,0,0,2,1,0,...,0,0.93641,0.910504,6,4,3.14815,2.16667,3,2.84091,3
10684,20112012,2,3,0,2,4,0,5,4,0,...,0,0.905153,0.925626,26,3,2.77273,2.54545,2.45238,2.35714,5
10477,20112012,4,3,0,2,1,0,1,2,0,...,0,0.901833,0.895013,17,9,3.19565,3.08696,2.33333,3.25641,8
9312,20112012,4,0,1,4,3,0,1,4,0,...,0,0.895866,0.920074,29,18,2.65854,3.07317,2.5,2.45455,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,20172018,6,5,1,0,1,1,3,4,0,...,0,0.932634,0.917073,15,14,3.125,1.83333,2.58537,2.65854,6
634,20172018,6,5,1,0,1,1,3,4,0,...,0,0.932634,0.917073,15,14,3.125,1.83333,2.58537,2.65854,6
635,20172018,3,0,1,5,6,0,1,0,0,...,0,0.902329,0.909422,14,15,3.12195,2.87805,3.17021,2.76596,5
636,20172018,6,5,1,0,1,1,3,4,0,...,0,0.932634,0.917073,15,14,3.125,1.83333,2.58537,2.65854,3


In [19]:
dataSet2TrainX = dataTrain1.drop('season',1).drop('total',1)
dataSet2TrainY = dataTrain1.total
dataSet2TestX = dataTest1.drop('season',1).drop('total',1)
dataSet2TestY = dataTest1.total

In [20]:
dataSet2TrainX

,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,game4home,...,game5away,game5isHome,home_perc,away_perc,home_team_id,away_team_id,home_scored_avg,home_conceded_avg,away_scored_avg,away_conceded_avg
11049,0,2,0,1,4,0,5,4,0,3,...,6,0,0.908803,0.915399,10,8,2.68293,2.90244,2.53488,2.65116
9192,1,2,1,3,0,0,2,1,0,6,...,5,0,0.93641,0.910504,6,4,3.14815,2.16667,3,2.84091
10684,2,3,0,2,4,0,5,4,0,4,...,4,0,0.905153,0.925626,26,3,2.77273,2.54545,2.45238,2.35714
10477,4,3,0,2,1,0,1,2,0,2,...,5,0,0.901833,0.895013,17,9,3.19565,3.08696,2.33333,3.25641
9312,4,0,1,4,3,0,1,4,0,3,...,2,0,0.895866,0.920074,29,18,2.65854,3.07317,2.5,2.45455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,6,5,1,0,1,1,3,4,0,3,...,3,0,0.932634,0.917073,15,14,3.125,1.83333,2.58537,2.65854
634,6,5,1,0,1,1,3,4,0,3,...,3,0,0.932634,0.917073,15,14,3.125,1.83333,2.58537,2.65854
635,3,0,1,5,6,0,1,0,0,4,...,4,0,0.902329,0.909422,14,15,3.12195,2.87805,3.17021,2.76596
636,6,5,1,0,1,1,3,4,0,3,...,3,0,0.932634,0.917073,15,14,3.125,1.83333,2.58537,2.65854


In [21]:
dataSet2TrainY

11049    2
9192     3
10684    5
10477    8
9312     5
        ..
633      6
634      6
635      5
636      3
637      4
Name: total, Length: 8377, dtype: object

In [22]:
dataSet2TrainX.to_csv('.\datasets\dataSet2TrainX.csv', encoding='utf-8', index=False)
dataSet2TrainY.to_csv('.\datasets\dataSet2TrainY.csv', encoding='utf-8',index=False)
dataSet2TestX.to_csv('.\datasets\dataSet2TestX.csv', encoding='utf-8', index=False)
dataSet2TestY.to_csv('.\datasets\dataSet2TestY.csv', encoding='utf-8', index=False)

C:\Users\Filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\Filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.
